In [12]:
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import losses

In [8]:
# 导入数据集
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data('/home/zhouchen/Jupyter/卷积神经网络/data/mnist.npz')

In [9]:
# 调整训练数据为单通道图像输入
img_rows, img_cols = X_train[0].shape[0], X_train[0].shape[1]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [10]:
# 标准化数据
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [11]:
# label进行one-hot编码
n_classes = len(set(y_train))
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [14]:
# 定义CNN结构
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 256)         0         
__________

In [17]:
# 设置超参数
batch_size = 128
n_epochs = 200
callbacks = [EarlyStopping(monitor='val_acc', patience=5)]

In [18]:
# 训练模型
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_split=0.2, callbacks=callbacks)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 185s 4ms/step - loss: 0.0507 - acc: 0.9845 - val_loss: 0.0351 - val_acc: 0.9902
Epoch 2/10
48000/48000 [==============================] - 208s 4ms/step - loss: 0.0424 - acc: 0.9874 - val_loss: 0.0293 - val_acc: 0.9908
Epoch 3/10
48000/48000 [==============================] - 193s 4ms/step - loss: 0.0373 - acc: 0.9882 - val_loss: 0.0343 - val_acc: 0.9907
Epoch 4/10
48000/48000 [==============================] - 157s 3ms/step - loss: 0.0350 - acc: 0.9891 - val_loss: 0.0297 - val_acc: 0.9911
Epoch 5/10
48000/48000 [==============================] - 150s 3ms/step - loss: 0.0307 - acc: 0.9901 - val_loss: 0.0266 - val_acc: 0.9932
Epoch 6/10
48000/48000 [==============================] - 144s 3ms/step - loss: 0.0271 - acc: 0.9919 - val_loss: 0.0252 - val_acc: 0.9935
Epoch 7/10
48000/48000 [==============================] - 147s 3ms/step - loss: 0.0238 - acc: 0.9929 - val_loss: 0.0274 - 

In [19]:
# 在测试集上显示结果
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss", score[0])
print("Test accuracy", score[1])

Test loss 0.02027289513110045
Test accuracy 0.9942
